In [3]:
import os
import requests
import pandas as pd
from datetime import datetime

import os, requests, pandas as pd, pathlib, datetime as dt
from tqdm.auto import tqdm

In [5]:
from dotenv import load_dotenv
load_dotenv()           

import os
TOKEN  = os.getenv("EVENTBRITE_PRIVATE")   
BASE    = "https://www.eventbriteapi.com/v3"
HEADERS = {"Authorization": f"Bearer {TOKEN}"}
assert TOKEN, "Добавьте EVENTBRITE_PRIVATE в .env"


In [7]:

YEAR_START = 2019
YEAR_END   = 2024
STATUS     = "ended"        # можно 'live', 'draft', 'started', 'all'
PAGE_SIZE  = 200             # макс 200
SLEEP      = 0.1             # пауза между запросами, чтобы не спамить API

# ----------------------------- 3.   ФУНКЦИИ --------------------------

def list_my_orgs() -> list[str]:
    """Возвращает все org_id, доступные текущему токену."""
    url      = f"{BASE}/users/me/organizations/"
    org_ids  = []
    params   = {"page_size": PAGE_SIZE}
    while True:
        resp = requests.get(url, headers=HEADERS, params=params)
        resp.raise_for_status()
        pj = resp.json()
        org_ids.extend(org["id"] for org in pj["organizations"])
        if not pj["pagination"]["has_more_items"]:
            break
        params = {"continuation": pj["pagination"]["continuation"]}
    return org_ids


def fetch_events_brief(org_id: str) -> list[dict]:
    """Быстро вытягивает события организации (без venue)."""
    params = {
        "status": STATUS,
        "page_size": PAGE_SIZE,
        "start_date.range_start": f"{YEAR_START}-01-01T00:00:00Z",
        # upper bound нельзя — фильтруем потом
    }
    url   = f"{BASE}/organizations/{org_id}/events/"
    items = []
    while True:
        r = requests.get(url, headers=HEADERS, params=params)
        r.raise_for_status()
        pj = r.json()
        items.extend(pj["events"])
        if not pj["pagination"]["has_more_items"]:
            break
        params = {"continuation": pj["pagination"]["continuation"]}
        time.sleep(SLEEP)
    return items


def enrich_event(event_id: str) -> dict:
    """Достаёт полный JSON события с expand=venue."""
    url = f"{BASE}/events/{event_id}/?expand=venue"
    r   = requests.get(url, headers=HEADERS)
    r.raise_for_status()
    return r.json()


def normalize(events_raw: list[dict]) -> pd.DataFrame:
    rows = []
    for ev in events_raw:
        venue = ev.get("venue", {})
        addr  = venue.get("address", {})
        start = ev["start"]["utc"]
        if start < f"{YEAR_START}-01-01T00:00:00Z" or start > f"{YEAR_END}-12-31T23:59:59Z":
            continue  # фильтруем верхнюю границу и лишние статусы
        rows.append({
            "event_id":  ev["id"],
            "org_id":    ev["organization_id"],
            "name":      ev["name"]["text"],
            "start_utc": start,
            "end_utc":   ev["end"]["utc"],
            "status":    ev.get("status"),
            "category":  ev.get("category_id"),
            "capacity":  ev.get("capacity"),
            "venue_id":  venue.get("id"),
            "lat":       addr.get("latitude"),
            "lon":       addr.get("longitude"),
            "address":   addr.get("localized_address_display"),
            "wkt": (f"POINT({addr.get('longitude')} {addr.get('latitude')})"
                     if addr.get("latitude") and addr.get("longitude") else None)
        })
    df = pd.DataFrame(rows)
    if not df.empty:
        df["start_utc"] = pd.to_datetime(df["start_utc"])
        df["end_utc"]   = pd.to_datetime(df["end_utc"])
        df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
        df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
    return df

# ----------------------------- 4.   MAIN ------------------------------

def main():
    org_ids = list_my_orgs()
    if not org_ids:
        raise SystemExit("У токена нет организаций. Создайте черновик или запросите OAuth-доступ.")

    print("Найдено организаций:", len(org_ids), "→", ", ".join(org_ids))

    enriched_events: list[dict] = []

    for org in tqdm(org_ids, desc="Организации"):
        brief_events = fetch_events_brief(org)
        if not brief_events:
            print(f"⚠️  В орг {org} нет событий под статусом '{STATUS}' и датами >= {YEAR_START}")
            continue
        for ev in tqdm(brief_events, desc=f"  events {org}", leave=False):
            try:
                enriched = enrich_event(ev["id"])
                enriched_events.append(enriched)
            except requests.HTTPError as e:
                print("🚫", ev["id"], e)
            time.sleep(SLEEP)

    if not enriched_events:
        raise SystemExit("Не найдено подходящих событий.")

    df = normalize(enriched_events)
    print("✅ Событий после фильтра:", len(df))

    outdir = pathlib.Path("eventbrite_exports")
    outdir.mkdir(exist_ok=True)
    outfile = outdir / f"events_all_{dt.datetime.utcnow():%Y%m%d}.csv"
    df.to_csv(outfile, index=False, encoding="utf-8")

    from IPython.display import display  # Jupyter preview
    display(df.head())
    print("📄 CSV сохранён:", outfile.resolve())


In [1]:
# -*- coding: utf-8 -*-
"""eventbrite_export_full.py

Автономный скрипт для Jupyter / Python 3.x
-------------------------------------------
* Берёт PRIVATE‑токен из .env (EVENTBRITE_PRIVATE)
* Находит все организации, к которым токен имеет доступ
* Выгружает события (ended/live/draft — задаётся параметром) за диапазон лет
* Для каждого события делает отдельный запрос с expand=venue, чтобы получить lat/lon
* Сохраняет результат в CSV + выводит превью DataFrame

Зависимости (pip install): requests pandas python-dotenv tqdm
"""

# ----------------------------- 1. ИМПОРТЫ -----------------------------
from __future__ import annotations
import os, pathlib, datetime as dt, time

import requests
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm

# ----------------------------- 2.   НАСТРОЙКИ ------------------------
load_dotenv()
TOKEN = os.getenv("EVENTBRITE_PRIVATE")
assert TOKEN, "⚠️  В .env нет EVENTBRITE_PRIVATE"

BASE    = "https://www.eventbriteapi.com/v3"
HEADERS = {"Authorization": f"Bearer {TOKEN}"}

YEAR_START = 2019
YEAR_END   = 2024
STATUS     = "ended"        # можно 'live', 'draft', 'started', 'all'
PAGE_SIZE  = 200             # макс 200
SLEEP      = 0.1             # пауза между запросами, чтобы не спамить API

# ----------------------------- 3.   ФУНКЦИИ --------------------------

def list_my_orgs() -> list[str]:
    """Возвращает все org_id, доступные текущему токену."""
    url      = f"{BASE}/users/me/organizations/"
    org_ids  = []
    params   = {"page_size": PAGE_SIZE}
    while True:
        resp = requests.get(url, headers=HEADERS, params=params)
        resp.raise_for_status()
        pj = resp.json()
        org_ids.extend(org["id"] for org in pj["organizations"])
        if not pj["pagination"]["has_more_items"]:
            break
        params = {"continuation": pj["pagination"]["continuation"]}
    return org_ids


def fetch_events_brief(org_id: str) -> list[dict]:
    """Быстро вытягивает события организации (без venue)."""
    params = {
        "status": STATUS,
        "page_size": PAGE_SIZE,
        "start_date.range_start": f"{YEAR_START}-01-01T00:00:00Z",
        # upper bound нельзя — фильтруем потом
    }
    url   = f"{BASE}/organizations/{org_id}/events/"
    items = []
    while True:
        r = requests.get(url, headers=HEADERS, params=params)
        r.raise_for_status()
        pj = r.json()
        items.extend(pj["events"])
        if not pj["pagination"]["has_more_items"]:
            break
        params = {"continuation": pj["pagination"]["continuation"]}
        time.sleep(SLEEP)
    return items


def enrich_event(event_id: str) -> dict:
    """Достаёт полный JSON события с expand=venue."""
    url = f"{BASE}/events/{event_id}/?expand=venue"
    r   = requests.get(url, headers=HEADERS)
    r.raise_for_status()
    return r.json()


def normalize(events_raw: list[dict]) -> pd.DataFrame:
    rows = []
    for ev in events_raw:
        venue = ev.get("venue", {})
        addr  = venue.get("address", {})
        start = ev["start"]["utc"]
        if start < f"{YEAR_START}-01-01T00:00:00Z" or start > f"{YEAR_END}-12-31T23:59:59Z":
            continue  # фильтруем верхнюю границу и лишние статусы
        rows.append({
            "event_id":  ev["id"],
            "org_id":    ev["organization_id"],
            "name":      ev["name"]["text"],
            "start_utc": start,
            "end_utc":   ev["end"]["utc"],
            "status":    ev.get("status"),
            "category":  ev.get("category_id"),
            "capacity":  ev.get("capacity"),
            "venue_id":  venue.get("id"),
            "lat":       addr.get("latitude"),
            "lon":       addr.get("longitude"),
            "address":   addr.get("localized_address_display"),
            "wkt": (f"POINT({addr.get('longitude')} {addr.get('latitude')})"
                     if addr.get("latitude") and addr.get("longitude") else None)
        })
    df = pd.DataFrame(rows)
    if not df.empty:
        df["start_utc"] = pd.to_datetime(df["start_utc"])
        df["end_utc"]   = pd.to_datetime(df["end_utc"])
        df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
        df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
    return df

# ----------------------------- 4.   MAIN ------------------------------

def main():
    org_ids = list_my_orgs()
    if not org_ids:
        raise SystemExit("У токена нет организаций. Создайте черновик или запросите OAuth-доступ.")

    print("Найдено организаций:", len(org_ids), "→", ", ".join(org_ids))

    enriched_events: list[dict] = []

    for org in tqdm(org_ids, desc="Организации"):
        brief_events = fetch_events_brief(org)
        if not brief_events:
            print(f"⚠️  В орг {org} нет событий под статусом '{STATUS}' и датами >= {YEAR_START}")
            continue
        for ev in tqdm(brief_events, desc=f"  events {org}", leave=False):
            try:
                enriched = enrich_event(ev["id"])
                enriched_events.append(enriched)
            except requests.HTTPError as e:
                print("🚫", ev["id"], e)
            time.sleep(SLEEP)

    if not enriched_events:
        raise SystemExit("Не найдено подходящих событий.")

    df = normalize(enriched_events)
    print("✅ Событий после фильтра:", len(df))

    outdir = pathlib.Path("eventbrite_exports")
    outdir.mkdir(exist_ok=True)
    outfile = outdir / f"events_all_{dt.datetime.utcnow():%Y%m%d}.csv"
    df.to_csv(outfile, index=False, encoding="utf-8")

    from IPython.display import display  # Jupyter preview
    display(df.head())
    print("📄 CSV сохранён:", outfile.resolve())

# ----------------------------- 5.   RUN -------------------------------
if __name__ == "__main__":
    main()


AssertionError: ⚠️  В .env нет EVENTBRITE_PRIVATE